# Training: Neuronales Netzwerk mit dem Tuner von Weights & Biases

In diesem Notebook wird ein neuronales Netzwerk mit dem [Tuner von Weights & Biases](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/tensorflow/Hyperparameter_Optimization_in_TensorFlow_using_W%26B_Sweeps.ipynb#scrollTo=crY5iadfKp_b) trainiert. Der Tuner von Weights & Biases ist ein Tool, das automatisiert die Hyperparameter eines Modells optimiert. In diesem Fall wird der Tuner verwendet, um die Batch Size als Hyperparameter zu optimieren. Dabei ist das Ziel den Validierungsfehler zu minimieren.

Zunächst werden alle nötigen Bibliotheken importiert.

In [1]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import os
import keras_tuner as kt
import wandb
import tqdm
import tensorflow as tf
import keras
import seaborn as sns

from tensorflow.keras import layers, Model, optimizers, losses, callbacks
from tensorflow import GradientTape
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from wandb.integration.keras import WandbMetricsLogger
from sklearn.metrics import confusion_matrix

Anschließend können die verwendeten Datensätze geladen werden.

In [2]:
train = pd.read_csv('../data/preprocessed/train_data.csv')
test = pd.read_csv('../data/preprocessed/test_data.csv')
val = pd.read_csv('../data/preprocessed/val_data.csv')

In [3]:
train.shape, test.shape, val.shape

((50176, 48), (5469, 48), (10752, 48))

In [4]:
X_train = train.drop("Is high risk", axis=1)
Y_train = train["Is high risk"]

X_test = test.drop("Is high risk", axis=1)
Y_test = test["Is high risk"]

X_val = val.drop("Is high risk", axis=1)
Y_val = val["Is high risk"]

In [5]:
# wandb.login()

Nun werden die Funktionen für das neuronale Netzwerk definiert. Dabei wird ein einfaches Netzwerk mit vier versteckten Schichten mit jeweils 64 Neuronen und der ReLU-Aktivierungsfunktion verwendet. Weiterhin wird die Funktion für die Trainingsschritte und die Testschritte definiert.

In [6]:
def Model():
    x = layers.Input(shape=(X_train.shape[1],))

    h = layers.Dense(64, activation='relu')(x)
    h = layers.Dense(64, activation='relu')(h)
    h = layers.Dense(64, activation='relu')(h)
    h = layers.Dense(64, activation='relu')(h)

    y = layers.Dense(1, activation='sigmoid')(h)

    model = keras.Model(inputs=x, outputs=y)

    return model

def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
    with GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, logits)

    return loss_value

def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)

    return loss_value

Basierend auf den zuvor erstellten Funktionen wird nun die Trainingsfunktion erstellt. Dabei wird Weight & Biases verwendet, um den Trainingsprozess zu tracken.

In [7]:
def train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=1000, 
          log_step=200, 
          val_log_step=50):
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []   
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in tqdm.tqdm(enumerate(train_dataset), total=len(train_dataset)):
            loss_value = train_step(x_batch_train, y_batch_train, 
                                    model, optimizer, 
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value = test_step(x_batch_val, y_batch_val, 
                                       model, loss_fn, 
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))
            
        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_state()
        val_acc_metric.reset_state()

        # log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': float(train_acc), 
                   'val_loss': np.mean(val_loss),
                   'val_acc':float(val_acc)})

Nun werden die Parameter für die optimierung in der `sweep_config` definiert.

In [8]:
sweep_config = {
  'method': 'grid', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'values': [32, 64, 128, 256, 512]
      }
  }
}

Anschließend wird die `sweep_train`-Funktion erstellt. Diese nutzt die zuvor definierte Trainingsfunktion und führt sie mit der `swep_config` zusammen, um den Optimierungsprozess in sogenannten Sweeps durchzuführen und auf Weight & Biases zu tracken.

In [9]:
def sweep_train(config_defaults=None):
    # Set default values
    config_defaults = {
        "batch_size": 32,
        "learning_rate": 0.001
    }
    # Initialize wandb with a sample project name
    wandb.init(reinit = True, config=config_defaults)  # this gets over-written in the Sweep

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 10
    wandb.config.log_step = 20
    wandb.config.val_log_step = 50
    wandb.config.architecture_name = "test"
    wandb.config.dataset_name = "Credit Card"

    # build input pipeline using tf.data
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
    train_dataset = (train_dataset.batch(wandb.config.batch_size)
                                  .prefetch(buffer_size=tf.data.AUTOTUNE)
                                  .shuffle(buffer_size=50176)
                                  )

    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))
    val_dataset = (val_dataset.batch(wandb.config.batch_size)
                              .prefetch(buffer_size=tf.data.AUTOTUNE)
                              .shuffle(buffer_size=10752)
                              )

    # initialize model
    model = Model()

    # Instantiate an optimizer to train the model.
    # optimizer = keras.optimizers.SGD(learning_rate=wandb.config.learning_rate)
    optimizer = optimizers.Adam(learning_rate=wandb.config.learning_rate, 
                            beta_1=0.9, 
                            beta_2=0.999, 
                            amsgrad=True)
    # Instantiate a loss function.
    loss_fn = keras.losses.BinaryCrossentropy(from_logits=False)

    # Prepare the metrics.
    train_acc_metric = keras.metrics.BinaryAccuracy()
    val_acc_metric = keras.metrics.BinaryAccuracy()

    train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=wandb.config.epochs, 
          log_step=wandb.config.log_step, 
          val_log_step=wandb.config.val_log_step)

Nun wird ein neuer Sweep erstellt und die `sweep_train`-Funktion aufgerufen, um den Optimierungsprozess zu starten.

In [10]:
sweep_id = wandb.sweep(sweep_config, project="credit_card")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: gsbe8kka
Sweep URL: https://wandb.ai/lokrau/Credit_Card/sweeps/gsbe8kka


In [11]:
wandb.agent(sweep_id, function=sweep_train, count=5)

wandb: Agent Starting Run: h3pp5mfe with config:
wandb: 	batch_size: 32
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lokrau2002 (lokrau). Use `wandb login --relogin` to force relogin



Start of epoch 0


100%|██████████| 1568/1568 [00:31<00:00, 49.10it/s]
2024-06-09 11:50:33.789813: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training acc over epoch: 0.5190
Validation acc: 0.5007

Start of epoch 1


100%|██████████| 1568/1568 [00:30<00:00, 51.12it/s]
2024-06-09 11:51:05.329634: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

100%|██████████| 98/98 [00:02<00:00, 46.23it/s]
2024-06-09 12:01:21.515830: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training acc over epoch: 0.4709
Validation acc: 0.5394

Start of epoch 9


100%|██████████| 98/98 [00:02<00:00, 44.64it/s]
2024-06-09 12:01:23.803037: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training acc over epoch: 0.4853
Validation acc: 0.5032


acc,▄▃█▄▇█▁▅▃▄
epochs,▁▂▃▃▄▅▆▆▇█
loss,█▄▅▄▄▂▃▂▂▁
val_acc,▂▄▃▃▃▁▃▃█▃
val_loss,▂▃▂▆█▂▁▂▁▁
acc,0.48527
epochs,9
loss,22.10692
val_acc,0.50316
val_loss,21.5105


Nun werden die Ergebnisse dieses Sweeps geladen.

In [12]:
api = wandb.Api()
sweep_id = 'lokrau/credit_card/sweeps/' + sweep_id

sweep = api.sweep(sweep_id)
runs = sweep.runs

Diese ergebnisse werden in einer Liste gespeichert.

In [13]:
results = []
for run in runs:
    run_data = {
        'batch_size': run.config.get('batch_size'),
        'val_accuracy': run.summary.get('val_acc'),
        'val_loss': run.summary.get('val_loss'),
        'accuracy': run.summary.get('acc'),
        'loss': run.summary.get('loss'),
    }
    results.append(run_data)

In [14]:
results_df = pd.DataFrame(results)
results_df

,batch_size,val_accuracy,val_loss,accuracy,loss
0,512,0.503162,21.510502,0.485272,22.106925
1,256,0.507068,3.136579,0.566606,6.582804
2,128,0.498977,1.185536,0.539860,1.883175
3,64,0.521484,0.814311,0.556920,0.817636
4,32,0.504836,1.088050,0.543547,1.024513


Nun kann daraus der beste Run und damit die beste Batch Size ermittelt werden.

In [15]:
best_run = results_df.loc[results_df['val_loss'].idxmin()]
best_batch_size = best_run['batch_size']
best_val_loss = best_run['val_loss']

In [16]:
print(f'Best batch size: {best_batch_size} with validation loss: {best_val_loss}')

Best batch size: 64.0 with validation loss: 0.8143106691894078


Basierend darauf kann nun das neuronale Netzwerk mit der besten Batch Size trainiert werden.

In [17]:
model = Model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 47)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,617 (61.00 KB)

 Trainable params: 15,617 (61.00 KB)

 Non-trainable params: 0 (0.00 B)

Dabei wird mithilfe des Validierungsdatensatzes ein Early Stopping implementiert, um das Training zu stoppen, wenn der Validierungsfehler nicht mehr sinkt.

In [18]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    min_delta=0.001,     # Minimum change to qualify as an improvement
    patience=100,         # How many epochs to wait after last time val loss improved
    verbose=1,
    mode='min',          # The training will stop when the quantity monitored has stopped decreasing
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity.
)

Als Optimizer wird der Adam-Optimizer verwendet.

In [19]:
optimizer = optimizers.Adam(learning_rate=0.001, 
                            beta_1=0.9, 
                            beta_2=0.999, 
                            amsgrad=True)

model.compile(optimizer=optimizer, 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
batch_size = int(best_batch_size)
epochs = 1000

Nun wird Weight & Biases initialisiert, um den Trainingsprozess zu tracken.

In [21]:
# wandb.login()

Dazu wird ein neuer Run erstellt.

In [22]:
run = wandb.init(project='ml_assignment_wandb',
                 reinit = True,
                 config={
                     "learning_rate": 0.001, 
                     "epochs": epochs,
                     "batch_size": batch_size,
                     "loss_function": "BinaryCrossentropy",
                     "architecture": "Dense(64, relu) -> Dense(64, relu) -> Dense(64, relu) -> Dense(64, relu) -> Dense(1, sigmoid)",
                     "dataset": "Credit Card",
                     "optimizer": "Adam"})

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Anschließend kann das Training gestartet werden und der Trainingsprozess auf Weight & Biases mithilfe des `WandbMetricsLogger` getrackt werden.

In [23]:
history = model.fit(X_train, 
                    Y_train, 
                    batch_size=batch_size,
                    validation_data=(X_val, Y_val),
                    callbacks=[early_stopping, WandbMetricsLogger()],
                    epochs=epochs)

if not os.path.exists('../models'):
    os.makedirs('../models')

model.save('../models/model_wandb_tuning.h5')

Epoch 1/1000
729/784 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - accuracy: 0.5130 - loss: 293.5122

wandb: ERROR Unable to log learning rate.


784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 879us/step - accuracy: 0.5129 - loss: 281.1063 - val_accuracy: 0.5037 - val_loss: 40.6841
Epoch 2/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.5227 - loss: 38.7160 - val_accuracy: 0.5040 - val_loss: 11.6041
Epoch 3/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 787us/step - accuracy: 0.5215 - loss: 12.9736 - val_accuracy: 0.5107 - val_loss: 9.6152
Epoch 4/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.5148 - loss: 8.9056 - val_accuracy: 0.5193 - val_loss: 3.3160
Epoch 5/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 760us/step - accuracy: 0.5252 - loss: 5.1333 - val_accuracy: 0.5073 - val_loss: 5.8408
Epoch 6/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.5339 - loss: 3.4577 - val_accuracy: 0.5086 - val_loss: 2.0789
Epoch 7/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 779us/step - accuracy: 0.5311 - loss: 3.2955 - val_accuracy: 0.5101 - val_loss: 4.4380
Epoch 8/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step - accuracy: 0.5494 - loss: 

KeyboardInterrupt: 

wandb: Network error (ConnectionError), entering retry loop.


In [ ]:
wandb.finish()

Nun wird der Trainingsverlauf des Modells für Accuracy und Loss visualisiert.

In [ ]:
# Print the model training history for accuracy
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(accuracy) + 1)

plt.figure(figsize=(10,8))
plt.plot(epochs, accuracy)
plt.plot(epochs, val_accuracy)
plt.title('Model Accuracy', fontsize=15, pad=15)
plt.ylabel('Accuracy', fontsize=12, labelpad=10)
plt.xlabel('Epoche', fontsize=12, labelpad=10)
plt.legend(['Training', 'Validation'], loc='lower right')
plt.show()

In [ ]:
# print the model training history for loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.figure(figsize=(10,8))
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Model Loss', fontsize=15, pad=15)
plt.ylabel('Loss', fontsize=12, labelpad=10)
plt.xlabel('Epoche', fontsize=12, labelpad=10)
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

Anschließend wird das Modell auf die drei zur verfügung stehenden Datensätze evaluiert und es ist zu erkennen, dass die Ergebnisse auf die drei Datensätze sehr ähnlich sind.

In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
model.evaluate(X_val, Y_val)

In [ ]:
model.evaluate(X_train, Y_train)

Abschließend wird noch eine Confusion Matrix für die Testdaten erstellt, um die Ergebnisse des Modells zu visualisieren.

In [ ]:
# make a confusion matrix for the test data
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

cm = confusion_matrix(Y_test, y_pred)

# Set the style of the heatmap
sns.set_theme(style="whitegrid")

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', square=True, cmap='Blues', cbar_kws={'shrink': 0.75}, linewidths=1, linecolor='grey')

# Add labels
plt.xlabel('Predicted', fontsize=12, labelpad=10)
plt.ylabel('True', fontsize=12, labelpad=10)
plt.title('Confusion Matrix', fontsize=15, pad=15)

# Improve the layout
plt.tight_layout()

# Display the plot
plt.show()
